##  RAG Pipeline

In [25]:
from langgraph.graph import StateGraph, END, START
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from typing import TypedDict, Annotated
from pydantic import BaseModel, Field
import operator
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_community.vectorstores import FAISS
from pathlib import Path
from langchain_community.document_loaders import PyPDFLoader, TextLoader, UnstructuredWordDocumentLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
load_dotenv()
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

In [43]:
embedd = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [9]:
llm.invoke("hii").content

'Hi there! How can I help you today?'

In [55]:
def read_docs(folder_name):
    all_docd = []

    for i in Path(folder_name).glob("*"):
        if i.suffix == ".pdf":
            loader = PyPDFLoader(str(i))
        elif i.suffix == ".text":
            loader = TextLoader(i,"utf-8")
        elif i.suffix == ".docx":
            loader = UnstructuredWordDocumentLoader(i,"utf-8")    
        else:
            pass

        docs = loader.load()
        all_docd.extend(docs)
    return all_docd

In [56]:
folder_name = "docss"

In [57]:
documents = read_docs(folder_name=folder_name)

In [58]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 100,chunk_overlap=50)
chunks = text_splitter.split_documents(documents=documents)

In [59]:
vector_store = FAISS.from_documents(chunks,embedd)
retriever = vector_store.as_retriever(search_type="similarity",kwargs={"k":5})


In [60]:
question = "what is primary key"
retriever_docs = retriever.invoke(question)
context = "".join([i.page_content for i in retriever_docs])
print(context)

Primarykey:Aprimarykeyisafieldinatablethatuniquelyidentifieseachrow/recordinadatabasetable.PrimarykForeignKey:AforeignkeyisacolumnoracombinationofcolumnswhosevaluesmatchaPrimaryKeyinadifferenttable.selfconsideringemp_idastheprimarykeyandmanager_idastheforeignkey.Query:
